In [1]:
import pandas as pd
train = pd.read_csv('../input/word2vec-nlp-tutorial/labeledTrainData.tsv', header=0,delimiter="\t")
test = pd.read_csv('../input/word2vec-nlp-tutorial/testData.tsv', header=0, delimiter="\t")

In [2]:
train.head(1)

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...


In [3]:
train['review'][0]

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

In [4]:
import re
from bs4 import BeautifulSoup 
def review_to_wordlist(review):
    '''
    Meant for converting each of the IMDB reviews into a list of words.
    '''
    # First remove the HTML.
    review_text = BeautifulSoup(review).get_text()
    
    # Use regular expressions to only include words.
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    
    # Convert words to lower case and split them into separate words.
    words = review_text.lower().split()
   
    # Return a list of words
    return(words)

In [5]:
%%time
traindata = []
for i in range(0,len(train['review'])):
    traindata.append(" ".join(review_to_wordlist(train['review'][i])))
testdata = []
for i in range(0,len(test['review'])):
    testdata.append(" ".join(review_to_wordlist(test['review'][i])))

E:\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file E:\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


### word2vec  词的向量化

In [6]:
%%time
from sklearn.feature_extraction.text import TfidfVectorizer
tfv = TfidfVectorizer(stop_words = 'english')

X_all = traindata + testdata 

tfv.fit(X_all)
X_all = tfv.transform(X_all)

Wall time: 16 s


In [7]:
lentrain = len(traindata)
X_train = X_all[:lentrain]  
X_test = X_all[lentrain:]

y_train = train['sentiment']

### 上算法

In [8]:
%%time
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()
log_reg.fit(X_train,y_train)

Wall time: 1.01 s


In [9]:
predict_result = log_reg.predict(X_test)
result_df = pd.DataFrame(data={"id":test["id"], "sentiment":predict_result})
result_df.to_csv("log_reg.csv",index=False)
    

In [10]:
%%time
#交叉验证
from sklearn.model_selection import cross_val_score, StratifiedKFold
kfold = StratifiedKFold()

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import GridSearchCV
extra_trees = ExtraTreesClassifier(random_state=666)
ex_param_grid = {"max_depth": [None],
              #"max_features": [3,5,7,10],
              #"min_samples_split": [2, 3, 10],
              #"min_samples_leaf": [1, 3, 10],
              "bootstrap": [False],
              "n_estimators" :[100,300,500,1000],
              "criterion": ["gini"]}
extra_trees_grid_search = GridSearchCV(extra_trees,param_grid = ex_param_grid, cv=kfold, scoring="accuracy", n_jobs=-1, verbose = 1)
extra_trees_grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Done  10 out of  12 | elapsed: 33.8min remaining:  6.8min
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed: 37.2min finished


Wall time: 1h 5min 43s


In [12]:
extra_trees_best = extra_trees_grid_search.best_estimator_
extra_trees_best_score = extra_trees_grid_search.best_score_ 
predict_result = extra_trees_best.predict(X_test)
result_df = pd.DataFrame(data={"id":test["id"], "sentiment":predict_result})
result_df.to_csv("extra_tr.csv",index=False)